In [10]:
import gym
from gym import Env
import os
import random
from gym.spaces import Box,Discrete,Tuple,MultiBinary,MultiDiscrete,Dict
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [11]:
Discrete(3).sample()

0

In [12]:
Box(0,1,shape=(1,))

Box(1,)

In [13]:
Tuple((Discrete(3),Box(0,1,shape=(1,))))

Tuple(Discrete(3), Box(1,))

In [14]:
Dict({'height':Discrete(3),'width':Box(0,1,shape=(1,))})

Dict(height:Discrete(3), width:Box(1,))

In [15]:
MultiBinary(4).sample()

array([1, 0, 1, 1], dtype=int8)

In [16]:
MultiDiscrete([4,8,9]).sample()


array([0, 4, 1], dtype=int64)

In [17]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state
        

In [18]:
env=ShowerEnv()

In [19]:
env.observation_space.sample()

array([64.952705], dtype=float32)

In [20]:
env.reset()

array([39.])

In [22]:
from stable_baselines3.common.env_checker import check_env

In [23]:
check_env(env,warn=True)

In [25]:
episodes=5
for episode in range(1,episodes+1):
    obs=env.reset()
    done=False
    score=0
    if not done:
        env.render()
        action=env.action_space.sample()
        state,reward,info,done=env.step(action)
        reward+=score
        print('episode:{} score:{}'.format(episode,score))
        

episode:1 score:0
episode:2 score:0
episode:3 score:0
episode:4 score:0
episode:5 score:0


In [26]:
log_path=os.path.join('training','log_path')

In [28]:
model=PPO('MlpPolicy',env,verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [34]:
model.learn(total_timesteps=40000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -26.1    |
| time/              |          |
|    fps             | 2373     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -24         |
| time/                   |             |
|    fps                  | 1574        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007365668 |
|    clip_fraction        | 0.0433      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.985      |
|    explained_variance   | -1.39e-05   |
|    learning_rate        | 0.

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -19          |
| time/                   |              |
|    fps                  | 1250         |
|    iterations           | 11           |
|    time_elapsed         | 18           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0032218872 |
|    clip_fraction        | 0.0112       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.842       |
|    explained_variance   | 0.00139      |
|    learning_rate        | 0.0003       |
|    loss                 | 49.7         |
|    n_updates            | 150          |
|    policy_gradient_loss | 0.000432     |
|    value_loss           | 94.7         |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

In [36]:
evaluate_policy(model,env,n_eval_episodes=10)

G:\img_dect\tfod1\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-12.0, 58.787753826796276)